# Transfer learning met PyTorch

In deze notebook gaan we het concept van transfer learning onderzoeken. Transfer learning is een techniek waarbij een pre-trained neuraal netwerk wordt gebruikt als startpunt voor een nieuwe taak. Er zijn twee belangrijke benaderingen binnen transfer learning:

1. **Feature Extractie**: Hierbij houden we de gewichten van het pre-trained model vast en gebruiken we het model als feature extractor. Alleen het laatste (geclassificeerde) deel van het netwerk wordt aangepast aan de nieuwe taak.
  
2. **Fine-Tuning**: Hierbij worden de gewichten van (een deel van) het pre-trained model verder getraind op de nieuwe taak. Dit maakt het mogelijk om het model verder aan te passen aan de specifieke data.

## Stappen in de notebook

1. **Data voorbereiding**:
   - We gebruiken een kleine dataset voor classificatie (bijvoorbeeld een subset van de CIFAR-10 dataset).

2. **Model laden en voorbereiden**:
   - We laden een pre-trained model (bijv. ResNet-18) uit PyTorch's `torchvision` bibliotheek.
   - We maken aanpassingen voor beide benaderingen van transfer learning.

3. **Training**:
   - We demonstreren zowel de **feature extractie** als de **fine-tuning** methode.
   - We trainen de laatste laag voor de feature extractie en een deel van het netwerk voor fine-tuning.

4. **Evaluatie**:
   - We evalueren de prestaties van beide benaderingen op een testset.

---

## 1. Data Voorbereiding

We beginnen met het importeren van de benodigde bibliotheken en het laden van de dataset.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import random

# Data transformaties
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 pixels (resize 256 en de center crop tegelijkertijd gedaan)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# de preprocessing stappen kan je halen uit de beschrijving van het model (bvb op de pytorch api
#transform = torchvision.models.ResNet18_Weights.IMAGENET1K_V1.transforms

# Laden van de CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
total_samples = len(trainset)
sample_size = int(0.01 * total_samples)
random_indices = random.sample(range(total_samples), sample_size)
trainset_reduced = Subset(trainset, random_indices)
trainloader = DataLoader(trainset_reduced, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_samples = len(testset)
sample_size = int(0.01 * total_samples)
random_indices = random.sample(range(total_samples), sample_size)
testset_reduced = Subset(testset, random_indices)

# Create a DataLoader for the reduced dataset
testloader = DataLoader(testset_reduced, batch_size=32, shuffle=True)

# Klassen in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 2. Model Laden en Voorbereiden

### A) Feature Extractie

In deze aanpak houden we de gewichten van het pre-trained model vast en passen we alleen de laatste laag aan.

In [9]:
import torchvision.models as models
# fe voor feature extraction
model_fe = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1) # je moet de weigths meegeven anders enkel de structuur en niet getrained

for inputs, labels in trainloader:
    print('pretrained output shape', model_fe(inputs).shape)
    break

for param in model_fe.parameters():
    param.requires_grad = False # deze laag wordt niet getrained

#print(model_ft)
# fc gedeelte onderaan bevat het fully-connected gedeelte -> dit gaan we vervangen
num_ftrs_in = model_fe.fc.in_features # 512
model_fe.fc = nn.Linear(num_ftrs_in, 10) # 10 want cifar10

for inputs, labels in trainloader:
    print('transfer learned output shape', model_fe(inputs).shape)
    break


pretrained output shape torch.Size([32, 1000])
transfer learned output shape torch.Size([32, 10])


### B) Fine-Tuning

In deze aanpak staan we toe dat sommige (of alle) gewichten verder worden getraind.

In [10]:
model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1) # je moet de weigths meegeven anders enkel de structuur en niet getrained

for inputs, labels in trainloader:
    print('pretrained output shape', model_ft(inputs).shape)
    break

for name, param in model_ft.named_parameters():
    # train het model enkel vanaf de layer 4 (laatste convolutionele blok)
    if "layer_4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False # deze laag wordt niet getrained

#print(model_ft)
# fc gedeelte onderaan bevat het fully-connected gedeelte -> dit gaan we vervangen
num_ftrs_in = model_ft.fc.in_features # 512
model_ft.fc = nn.Linear(num_ftrs_in, 10) # 10 want cifar10

for inputs, labels in trainloader:
    print('transfer learned output shape', model_ft(inputs).shape)
    break


pretrained output shape torch.Size([32, 1000])
transfer learned output shape torch.Size([32, 10])


## 3. Training

Hier zullen we beide modellen trainen, eerst het model voor feature extractie, gevolgd door het model voor fine-tuning.

### A) Feature Extractie Training

In [13]:
import time

criterion = nn.CrossEntropyLoss()
optimizer_fe = optim.Adam(model_fe.parameters(), lr=0.001)

num_epochs=5
model_fe.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer_fe.zero_grad()
        outputs = model_fe(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_fe.step()
        
        running_loss += loss.item()

    end_time = time.time()

    print(f"Epoch {epoch}/{num_epochs}: time = {end_time-start_time}, loss = {running_loss/len(trainloader)}")


Epoch 0/5: time = 11.83214259147644, loss = 2.156807690858841
Epoch 1/5: time = 7.916046380996704, loss = 1.7110102325677872
Epoch 2/5: time = 7.726578235626221, loss = 1.4230331107974052
Epoch 3/5: time = 7.842657804489136, loss = 1.2272974774241447
Epoch 4/5: time = 8.521524906158447, loss = 1.1023725494742393


### B) Fine-Tuning Training

In [14]:
import time

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

num_epochs=5
model_ft.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer_ft.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()
        
        running_loss += loss.item()

    end_time = time.time()

    print(f"Epoch {epoch}/{num_epochs}: time = {end_time-start_time}, loss = {running_loss/len(trainloader)}")


Epoch 0/5: time = 8.349033832550049, loss = 2.242882765829563
Epoch 1/5: time = 8.441033124923706, loss = 1.7592771798372269
Epoch 2/5: time = 8.326905250549316, loss = 1.4430682510137558
Epoch 3/5: time = 8.53514838218689, loss = 1.2416529953479767
Epoch 4/5: time = 8.847198009490967, loss = 1.1138600558042526


## 4. Evaluatie
Na het trainen evalueren we de prestaties van beide modellen op de testset.

### A) Evaluatie van het Feature Extractie Model

In [15]:
correct = 0
total = 0

model_fe.eval()

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model_fe(inputs)
        _, predictions = torch.max(outputs, 1) # maximale_waarden , indices
        total += labels.size(0) # batch_size
        correct += (predictions == labels).sum().item()

print('Accuracy is :', correct/total*100)

Accuracy is : 69.0


### B) Evaluatie van het Fine-Tuning Model

In [16]:
correct = 0
total = 0

model_ft.eval()

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model_ft(inputs)
        _, predictions = torch.max(outputs, 1) # maximale_waarden , indices
        total += labels.size(0) # batch_size
        correct += (predictions == labels).sum().item()

print('Accuracy is :', correct/total*100)

Accuracy is : 68.0


## Oefening: transfer learning met extra lagen voor FashionMNIST

### Oefeningomschrijving:
In deze oefening ga je een neuraal netwerk trainen met de FashionMNIST dataset, gebruikmakend van transfer learning. Je zult een pre-trained ResNet-18 model gebruiken, en daar drie extra volledig verbonden lagen aan toevoegen. Vervolgens train je het model en evalueer je de prestaties.

### Stappen:

**Data Voorbereiding:**

Laad de FashionMNIST dataset en breng de nodige transformaties aan.
Splits de data in een trainingsset en een testset.

**Model Voorbereiding:**

Laad een pre-trained ResNet-18 model.
Pas het model aan door drie extra volledig verbonden lagen toe te voegen:
* De eerste extra laag moet 512 neuronen hebben en een ReLU activatiefunctie.
* De tweede extra laag moet 256 neuronen hebben en een ReLU activatiefunctie.
* De derde extra laag moet 128 neuronen hebben en een ReLU activatiefunctie.
* De laatste output laag moet het aantal klassen in FashionMNIST (10 klassen) bevatten.

**Training:**

Definieer een loss-functie en een optimizer.
Train het model voor een aantal epochs.
Meet en print de tijd die elke epoch kost.

**Evaluatie:**

Evalueer de prestaties van het getrainde model op de testset en rapporteer de accuraatheid.

**Vragen om te beantwoorden:**

* Wat is het effect van de extra lagen op de prestaties van het model?
* Hoeveel tijd kost elke epoch en hoe varieert dit met het aantal lagen?

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST
from torchvision import models
from torch.utils.data import DataLoader, random_split
import time

# Transformaties voor de FashionMNIST dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet vereist 3 kanalen
    transforms.Resize((224, 224)),               # ResNet vereist input van 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# Laad de dataset en splits in train/test sets
dataset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Laad het pre-trained ResNet-18 model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
for param in model.parameters():
    param.requires_grad = False  # Freeze pre-trained lagen

# Pas de laatste lagen van het model aan
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 10)  # Output laag voor 10 klassen
)

# Definieer de loss-functie en optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training van het model
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    epoch_time = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Time: {epoch_time:.2f}s")

# Evaluatie van het model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')
